# Coding II -- Day 9 -- Regressions in research
-- Haynes Stephens -- haynes13@uchicago.edu

**NOTE**: If you're following along to a live lecture, don't worry about reading the text in this notebook. I'll summarize the text as we move forward, and it's there in case you need to go back and refresh your memory at a later time.

Today we will be working on creating regression models in Python and seeing how we can take advantage of using `R` and `Python` together. While this lesson will cover topics in statistics and machine learning, the purpose of the lesson is to show you the tools available in `Python`. I'm no expert on the statistics behind these methods and may not be able to answer a lot of your theoretical questions, but the bootcamp session next week will be more tailored towards the theory. 

## 0.1 Import required packages

Let's import some of the stuff we'll need off the bat.

In [ ]:
# Data analysis
import numpy as np
import pandas as pd
%load_ext google.colab.data_table

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objs as go
import plotly.express as px

Let's mount our drive so that we can have access to our Google Drive folders and files.

In [ ]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

### List the directories you'll use
loaddir = '/content/drive/Shared drives/Coding_Bootcamps_2020/computing_for_research/data/'
savedir = '/content/drive/My Drive/my_bootcamp_2020/'     

# Section 1: Introducing Scikit-Learn

## 1.1 Scikit-Learn's Estimator API

The Scikit-Learn API is designed with the following guiding principles in mind, as outlined in the [Scikit-Learn API paper](http://arxiv.org/abs/1309.0238):

- *Consistency*: All objects share a common interface drawn from a limited set of methods, with consistent documentation.

- *Inspection*: All specified parameter values are exposed as public attributes.

- *Limited object hierarchy*: Only algorithms are represented by Python classes; datasets are represented
  in standard formats (NumPy arrays, Pandas ``DataFrame``s, SciPy sparse matrices) and parameter
  names use standard Python strings.

- *Composition*: Many machine learning tasks can be expressed as sequences of more fundamental algorithms,
  and Scikit-Learn makes use of this wherever possible.

- *Sensible defaults*: When models require user-specified parameters, the library defines an appropriate default value.

In practice, these principles make Scikit-Learn very easy to use, once the basic principles are understood.
Every machine learning algorithm in Scikit-Learn is implemented via the Estimator API, which provides a consistent interface for a wide range of machine learning applications.

There are several Python libraries which provide solid implementations of a range of machine learning algorithms.
One of the best known is [Scikit-Learn](http://scikit-learn.org), a package that provides efficient versions of a large number of common algorithms.
Scikit-Learn is characterized by a clean, uniform, and streamlined API, as well as by very useful and complete online documentation.
A benefit of this uniformity is that once you understand the basic use and syntax of Scikit-Learn for one type of model, switching to a new model or algorithm is very straightforward.

This section provides an overview of the Scikit-Learn API; a solid understanding of these API elements will form the foundation for understanding the practical methods of machine learning.

We will start by covering *data representation* in Scikit-Learn, followed by covering the *Estimator* API, and finally go through a more interesting example of using these tools for exploring a set of images of hand-written digits.

In [ ]:
import sklearn
# we will load individual sklearn models and functions as we go along

## 1.2 Data Representation in Scikit-Learn

Machine learning is all about creating regression models from data: for that reason, we'll start by discussing how data can be represented in order to be understood by the computer.
The best way to think about data within `scikit-learn` is in terms of tabular data.


A basic table is a two-dimensional grid of data, in which the rows represent individual elements of the dataset, and the columns represent values related to each of these elements.
For example, here is a dataset I made up about how much water a person needs to drink depending on the average daily temperature.

In [ ]:
rng = np.random.RandomState(42) 
x = 70 * rng.rand(50) + 50
y = 0.25 * x - (rng.randn(50)*2) - 5
df = pd.DataFrame({"temperature":x, "cups_of_water":y})
df

Each row of the dataset refers to a single observation, and the number of rows is the total number of observations in the dataset.
In general, we will refer to the rows of the matrix as *samples*, and the number of rows as ``n_samples``.

Likewise, each column of the data refers to a particular quantitative piece of information that describes each sample.
In general, we will refer to the columns of the matrix as *features*, and the number of columns as ``n_features``. 

Here's a quick diagram to help show these structures.

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ax.axis('equal')

color = 'black' # change to white if you ARE in DARK MODE
# Draw features matrix
ax.vlines(range(6), ymin=0, ymax=9, lw=1, color=color)
ax.hlines(range(10), xmin=0, xmax=5, lw=1, color=color)
font_prop = dict(size=12, family='monospace', color=color)
ax.text(-1, -1, "Feature Matrix ($X$)", size=14, color=color)
ax.text(0.1, -0.3, r'n_features $\longrightarrow$', **font_prop)
ax.text(-0.1, 0.1, r'$\longleftarrow$ n_samples', rotation=90,
        va='top', ha='right', **font_prop)

# Draw labels vector
ax.vlines(range(8, 10), ymin=0, ymax=9, lw=1, color=color)
ax.hlines(range(10), xmin=8, xmax=9, lw=1, color=color)
ax.text(7, -1, "Target Vector ($y$)", size=14, color=color)
ax.text(7.9, 0.1, r'$\longleftarrow$ n_samples', rotation=90,
        va='top', ha='right', **font_prop,)

ax.set_ylim(10, -2)
plt.savefig(savedir+'regress_matricies.png')

### 1.2.1 Features matrix

This table layout makes clear that the information can be thought of as a two-dimensional numerical array or matrix, which we will call the *features matrix*.
By convention, this features matrix is often stored in a variable named ``X``.
The features matrix is assumed to be two-dimensional, with shape ``[n_samples, n_features]``, and is most often contained in a `numpy` array or a `pandas` ``DataFrame``.

The samples (i.e., rows) always refer to the individual objects described by the dataset.
For example, the sample might be a flower, a person, a document, an image, a sound file, a video, an astronomical object, or anything else you can describe with a set of quantitative measurements.

The features (i.e., columns) always refer to the distinct observations that describe each sample in a quantitative manner.
Features are generally real-valued, but may be Boolean or discrete-valued in some cases.

### 1.2.2 Target array

In addition to the feature matrix ``X``, we also generally work with a *label* or *target* array, which by convention we will usually call ``y``.
The target array is usually one dimensional, with length ``n_samples``, and is generally contained in a NumPy array or Pandas ``Series``.
The target array may have continuous numerical values, or discrete classes/labels.
We will be working with the common case of a one-dimensional target array.

A common point of confusion is how the target array differs from the other features columns. The distinguishing feature of the target array is that it is usually the quantity we want to *predict from the data*: in statistical terms, it is the dependent variable.
For example, in the preceding data we may wish to construct a model that can predict the cups of water that should be drank based on the daily temperature; in this case, the ``cups_of_water`` column would be considered the target array.

With this target array in mind, we can use Plotly to conveniently visualize the data:

## 1.3 Check-in 1

Let's visualize this data real quick to see how it looks. Use either `seaborn.relplot()` or `plotly_express.scatter()` to make a plot of `cups of water` vs. `temperature`. Feel free to check out the **Visualization** notebook to refresh your knowledge of plot commands.

In [ ]:
# WRITE YOUR CODE IN THIS CELL

### Answer

In [ ]:
fig = px.scatter(
    df, 
    x="temperature",
    y = "cups_of_water",
    width = 800,
    height = 400,
    template = 'plotly_dark',
    )
fig.show()

## 1.4 Basics of the Scikit's API

Most commonly, the steps in using the Scikit-Learn estimator API are as follows:

1. Choose a class of model.
2. Choose model hyperparameters.
3. Arrange data into a features matrix and target vector.
4. Fit the model to your data by calling the ``fit()`` method of the model instance.
5. Apply the model to new data using the ``predict()`` method.

We will now step through an example of applying these steps.

# Section 2: Linear Regression

We will start linear regression, where we assume that the target is related to the features by linear coefficients. The most familiar linear regression is a straight-line fit to data.
A straight-line fit is a model of the form
$$
y = wx + b
$$
where $w$ is commonly known as the *weight* (or slope), and $b$ is commonly known as the *bias* (or intercept).

## 2.1 Example

As an example of this process, let's consider a simple linear regression—that is, the common case of fitting a line to $(x, y)$ data.

Consider the following data, which is scattered about a line with a slope of 2 and an intercept of -5

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 5 + rng.randn(50)
plt.scatter(x, y)

### 2.1.1 Choose a class of model

In Scikit-Learn, every class of model is represented by a Python class.
So, for example, if we would like to compute a simple linear regression model, we can import the linear regression class:

In [ ]:
from sklearn.linear_model import LinearRegression

Note that other more general linear regression models exist as well; you can read more about them in the [``sklearn.linear_model`` module documentation](http://Scikit-Learn.org/stable/modules/linear_model.html).

### 2.1.2 Choose model hyperparameters

An important point is that once we have decided on our model **class**, there are still some options open to as to the **instance** of model we want to use.
Depending on the model class we are working with, we might need to answer one or more questions like the following:

- Would we like to fit for the offset (i.e., *y*-intercept)?
- Would we like the model to be normalized?
- Would we like to preprocess our data to add model flexibility?
- Would we like to prescribe regularization to use in our model?
- How many model components would we like to use?

These are examples of the important choices that must be made **once the model class is selected**.
These choices are often represented as **hyperparameters**, or parameters that must be set before the model is fit to data.
In Scikit-Learn, **hyperparameters** are chosen by passing values at model instantiation.

For our linear regression example, we can instantiate the `LinearRegression` class and specify that we would like to fit the intercept using the `fit_intercept` hyperparameter:

In [ ]:
model = LinearRegression(fit_intercept=True)
model

Keep in mind that when the model is instantiated, the only action is the storing of these hyperparameter values.
In particular, we have not yet applied the model to any data: the Scikit-Learn API makes very clear the distinction between **choosing the model** and **applying the model to data**.

### 2.1.3 Arrange data into a features matrix and target vector

Previously we detailed the Scikit-Learn data representation, which requires a two-dimensional features matrix and a one-dimensional target array.
Here our target variable ``y`` is already in the correct form (a length-``n_samples`` array), but we need to massage the data ``x`` to make it a matrix of size ``[n_samples, n_features]``.
In this case, this amounts to a simple reshaping of the one-dimensional array:

In [ ]:
x.shape

In [ ]:
X = x[:, np.newaxis]
X.shape

### 2.1.4 Fit the model to your data

Now it is time to apply our model to data.
This can be done with the ``fit()`` method of the model:

In [ ]:
model.fit(X, y)

This ``fit()`` command causes a number of model-dependent internal computations to take place, and the results of these computations are stored in attributes that the user can explore.
In Scikit-Learn, by convention all model parameters that were learned during the `fit()` process have trailing underscores. For example in this linear model, the slope and intercept of the model fit are referred to as the `coef_` array and the `intercept_` value:

In [ ]:
model.coef_, model.intercept_

These two parameters represent the slope and intercept of the simple linear fit to the data.

**Comparing to the data, we see that they are very close to the defined slope of 2 and intercept of -5.**

One question that frequently comes up regards the uncertainty in such internal model parameters.
In general, Scikit-Learn does not provide tools to draw conclusions from internal model parameters themselves: interpreting model parameters is much more a **statistical modeling** question than a **machine learning** question.
Machine learning rather focuses on what the model *predicts*.
If you would like to dive into the meaning of fit parameters within the model, other tools are available, including the [Statsmodels Python package](http://statsmodels.sourceforge.net/). 

### 2.1.5 Predict labels for unknown data

Once the model is trained, the main task of supervised machine learning is to evaluate it based on what it says about new data that was **not** part of the training set.
In Scikit-Learn, this can be done using the ``predict()`` method.
For the sake of this example, our "new data" will be a grid of *x* values, and we will ask what *y* values the model predicts:

In [ ]:
xfit = np.linspace(-1, 11)

As before, we need to coerce these *x* values into a ``[n_samples, n_features]`` features matrix, after which we can feed it to the model:

In [ ]:
Xfit = xfit[:, np.newaxis]
yfit = model.predict(Xfit)

Finally, let's visualize the results by plotting first the raw data, and then this model fit:

In [ ]:
plt.scatter(x, y)       # A scatter of the data points
plt.plot(xfit, yfit)    # The line our model predicts

## 2.2 Check-in 2

Going back to our **temperature-water** dataset:
1. Create a linear regression using Sci-KitLearn, where you fit *with* an intercept. Remember to massage the `x` data.
2. Plot the linear fit as a dashed line.
3. Print the slope and intercept values.

In [ ]:
# WRITE YOUR CODE IN THIS CELL

### 2.2.1 Answer

In [ ]:
model = LinearRegression(fit_intercept=True)

x = df["temperature"]
x = x[:, np.newaxis]
y = df["cups_of_water"]

#rearange x to be a 2D vertial array with np.newaxis (y array is 1D)
model.fit(x, y) 

xfit = np.linspace(50, 120, 1000)
xfit = xfit[:, np.newaxis]
yfit = model.predict(xfit)

plt.scatter(x, y)
plt.plot(xfit, yfit, linestyle='dashed')

print("Model slope:    ", model.coef_[0])
print("Model intercept:", model.intercept_)

## 2.3 EXTRA - Multiple regressors

The ``LinearRegression`` estimator is much more capable than this, however—in addition to simple straight-line fits, it can also handle multidimensional linear models of the form
$$
y = a_0 + a_1 x_1 + a_2 x_2 + \cdots
$$
where there are multiple $x$ values.
Geometrically, this is akin to fitting a plane to points in three dimensions, or fitting a hyper-plane to points in higher dimensions.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(100, 3)
poly = PolynomialFeatures(3, include_bias=False)
XN = poly.fit_transform(X)
XN.shape

In [ ]:
rng = np.random.RandomState(1)
X = 10 * rng.rand(100, 3)
y = 0.5 + np.dot(X, [1.5, -2., 1.])

model.fit(X, y)
print(model.intercept_)
print(model.coef_)

In [ ]:
df = pd.DataFrame({"X1":X[:, 0],
                  "X2":X[:, 1],
                  "X3":X[:, 2],
                  "Y":y})

fig = px.scatter_matrix(
    df, 
    dimensions=["X1", "X2", "X3", "Y"], 
    # color="species",
    width = 800,
    height = 400,
    opacity = 0.6,
    template = 'plotly_dark',
    )
fig.show()

Here the $y$ data is constructed from three random $x$ values, and the linear regression recovers the coefficients used to construct the data.

**Note**: The y variable shows a relationship with each of the X variables, and also important is that the X variables show no correlation to each other, a good sign. If our X variables were related then our model is probably hiding some influence from compounding variables.

In this way, we can use the single ``LinearRegression`` estimator to fit lines, planes, or hyperplanes to our data.
It still appears that this approach would be limited to strictly linear relationships between variables, but it turns out we can relax this as well.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
xfit = np.linspace(0, 10, 1000)
for i in range(len(axes)):
    ax = axes[i]
    ax.scatter(X[:, i], y)
    ax.plot(xfit, (xfit * model.coef_[i]) + model.intercept_)
    ax.set_title('$x_{0}$'.format(i))

## 2.4 Basis Functions

One trick you can use to adapt linear regression to nonlinear relationships between variables is to transform the data according to *basis functions*.

One type of basis function is called the Polynomial basis function.
The idea is to take our linear model:
$$
y = a_0 + a_1 x_1 + a_2 x_2 + a_3 x_3 + \cdots
$$
and build the $x_1, x_2, x_3,$ and so on, from our single-dimensional input $x$.
That is, we let $x_n = f_n(x)$, where $f_n()$ is some function that transforms our data.

For example, if $f_n(x) = x^n$, our model becomes a polynomial regression:
$$
y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots
$$
Notice that this is *still a linear model*—the linearity refers to the fact that the coefficients $a_n$ never multiply or divide each other.
What we have effectively done is taken our one-dimensional $x$ values and projected them into a higher dimension, so that a linear fit can fit more complicated relationships between $x$ and $y$.

This polynomial projection is useful enough that it is built into Scikit-Learn, using the ``PolynomialFeatures`` transformer:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

We see here that the transformer has converted our one-dimensional array into a three-dimensional array by taking the exponent of each value.
This new, higher-dimensional data representation can then be plugged into a linear regression. For instance, let's use a **7th-order polynomial** regression.

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
poly_model = make_pipeline(PolynomialFeatures(7), LinearRegression())

With this transform in place, we can use the linear model to fit much more complicated relationships between $x$ and $y$. 
For example, here is a sine wave with noise:

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

xfit = np.linspace(0,10)

poly_model.fit(x[:, np.newaxis], y)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit);

Our linear model, through the use of **7th-order polynomial basis functions**, can provide an excellent fit to this non-linear data!

# Section 3: Hyperparameters and Model Validation

In the previous section, we saw the basic recipe for applying a linear regression model:

1. Choose a class of model
2. Choose model hyperparameters
3. Fit the model to the training data
4. Use the model to predict targets for new data

The first two pieces of this—the choice of model and choice of hyperparameters—are perhaps the most important part of using these tools and techniques effectively.
In order to make an informed choice, we need a way to *validate* that our model and our hyperparameters are a good fit to the data.
While this may sound simple, there are some pitfalls that you must avoid to do this effectively.

## 3.1 Selecting the Best Model

Now we will go into a litte more depth on selecting the "best" model and hyperparameters.
These issues are some of the most important aspects of the practice of machine learning, and I find that this information is often glossed over in introductory machine learning tutorials.

Of core importance is the following question: *if our estimator is underperforming, how should we move forward?*
There are several possible answers:

- Use a more complicated/more flexible model
- Use a less complicated/less flexible model
- Gather more training samples (increase N)
- Gather data to add more features

The answer to this question can sometimes be counter-intuitive.
In particular, sometimes using a more complicated model will give worse results, and adding more training samples may not improve your results!
The ability to determine what steps will improve your model is what separates the successful machine learning practitioners from the unsuccessful.

## 3.2 The Bias-variance trade-off

Fundamentally, the question of "the best model" is about finding a sweet spot in the tradeoff between *bias* and *variance*.
Consider the following figure, which presents two regression fits to the same dataset:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

def make_data(N=30, err=0.8, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y


X, y = make_data()
xfit = np.linspace(-0.1, 1.0, 1000)[:, None]
model1 = PolynomialRegression(1).fit(X, y)      # first-order fit
model20 = PolynomialRegression(20).fit(X, y)    # twentieth-order fit

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

ax[0].scatter(X.ravel(), y, s=40)
ax[0].plot(xfit.ravel(), model1.predict(xfit), color='gray')
ax[0].axis([-0.1, 1.0, -2, 14])
ax[0].set_title('High-bias model: Underfits the data', size=14)

ax[1].scatter(X.ravel(), y, s=40)
ax[1].plot(xfit.ravel(), model20.predict(xfit), color='gray')
ax[1].axis([-0.1, 1.0, -2, 14])
ax[1].set_title('High-variance model: Overfits the data', size=14)

It is clear that neither of these models is a particularly good fit to the data, but they fail in different ways.

The model on the left attempts to find a straight-line fit through the data.
Because the data are intrinsically more complicated than a straight line, the straight-line model will never be able to describe this dataset well.
Such a model is said to *underfit* the data: that is, it does not have enough model flexibility to suitably account for all the features in the data; another way of saying this is that the model has high *bias*.

The model on the right attempts to fit a high-order polynomial through the data.
Here the model fit has enough flexibility to nearly perfectly account for the fine features in the data, but even though it very accurately describes the training data, its precise form seems to be more reflective of the particular noise properties of the data rather than the intrinsic properties of whatever process generated that data.
Such a model is said to *overfit* the data: that is, it has so much model flexibility that the model ends up accounting for random errors as well as the underlying data distribution; another way of saying this is that the model has high *variance*.


We can also visualize our data along with polynomial fits of several degrees:

In [ ]:
X_test = np.linspace(-0.1, 1.1, 500)[:, None]

plt.scatter(X.ravel(), y, color='black')
axis = plt.axis()
for degree in [1, 3, 5]:        # Try first-, third-, and fifth-order fits

    y_test = PolynomialRegression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label='degree={0}'.format(degree))
    
plt.xlim(-0.1, 1.0)
plt.ylim(-2, 12)
plt.legend(loc='best');

## 3.3 Check-in 3: interactive exercise

Play around with model over-fitting in this interactive plot. Write down what you see and your thoughts.
* What happens as you increase the order of the polynomial fit?
* Do you notice any particular sweet spots, where it looks like the goldilocks zone of capturing the data's behavior without overfitting?

In [ ]:
X, y = make_data()

xfit = np.linspace(-0.1, 1.0, 1000)[:, None]
df = pd.DataFrame()
for i in range (1,21):
    model = PolynomialRegression(i).fit(X, y)
    yfit = model.predict(xfit)
    dfn = pd.DataFrame({'xfit':xfit.ravel(), 'yfit': yfit,  'order' : i})
    df = pd.concat([df, dfn])

In [ ]:
fig = px.line(
    df,
    x = 'xfit', 
    y = 'yfit', 
    animation_frame = 'order',
    animation_group= 'order',
    template = 'plotly_dark',
    height = 600,
    width = 800,
    line_shape = 'spline' #key -- or else it will render very funny
    )

fig.add_trace(
    go.Scatter(
        x = X.ravel(),
        y = y,
        mode = 'markers',
        showlegend=False, 
        )
)
fig.show()

## 3.4 Thinking about model validation

In principle, model validation is very simple: after choosing a model and its hyperparameters, we can estimate how effective it is by applying it to some of the training data and comparing the prediction to the known value.

The following sections first show a naive approach to model validation and why it
fails, before exploring the use of holdout sets and cross-validation for more robust
model evaluation.

### 3.4.1 Model validation the wrong way

Let's demonstrate the naive approach to validation using the same `X,y` data that we saw in the previous section.

Let's create a 20-degree polynomial and see how well it captures the data using **root mean squared error (RMSE)**. The closer to zero, the better the model.

**Note**: You can also normalize your RMSE value (e.g. by the mean or standard deviation of `y` data) to compare across datasets. 



In [ ]:
from sklearn.metrics import mean_squared_error
X, y    = make_data()
model   = PolynomialRegression(20)
model.fit(X, y)
ypred   = model.predict(X)
rmse    = mean_squared_error(y, ypred, squared=False)
print("RMSE = " + str(rmse))

We see a very low normalized RMSE (less than 1), which indicates that the model predicts our data very well!
But have we really come upon a model that we expect to be correct almost 100% of the time?

As you may have gathered, the answer is no.
In fact, this approach contains a fundamental flaw: *it trains and evaluates the model on the same data*.

What happens if we only train on the first half of the data, but predict on all the data?

In [ ]:
model = PolynomialRegression(20)
model.fit(X[:X.size//2], y[:X.size//2]) # Fit on the first half of the data
ypred   = model.predict(X)
rmse    = mean_squared_error(y, ypred, squared=False)
print("RMSE = " + str(rmse))

Holy cow! Not doing nearly as hot as before **:(**

What about if we train on *every other* data point, and still predict back on the entire set? 

In [ ]:
model = PolynomialRegression(20)
model.fit(X[::2], y[::2])           # Fit on every other point of data
ypred   = model.predict(X)
rmse    = mean_squared_error(y, ypred, squared=False)
print("RMSE = " + str(rmse))

### 3.4.2 Model validation the right way: Holdout sets

So what can be done?
A better sense of a model's performance can be found using what's known as a *holdout set*: that is, we hold back some subset of the data from the training of the model, and then use this holdout set to check the model performance.
This splitting can be done using the ``train_test_split`` utility in Scikit-Learn. Now let's see how well a **third-degree polynomial** regression model performs when we only train on half of the data.

In [ ]:
from sklearn.model_selection import train_test_split
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model = PolynomialRegression(3) # Fit a 3rd-order model on half the data
model.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = model.predict(X2)
rmse    = mean_squared_error(y2, y2_model, squared=False)
print("RMSE = " + str(rmse))

We see here a much more reasonable result: the performance isn't as good as the 20th-order polynomial trained on the entire dataset, but it's much better at predicting on new data than the 20th-order model trained on only half of the data. 

The hold-out set is similar to unknown data, because the model has not "seen" it before.

## 3.5 Check -in 4

Using the code cell above, go through a series of degreed polynomials and find the "best" model performance based on a half-and-half split of the data.

In [ ]:
# WRITE YOUR CODE HERE

### Answer

In [ ]:
from sklearn.model_selection import train_test_split
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model = PolynomialRegression(5) # Fit a 3rd-order model on half the data
model.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = model.predict(X2)
rmse    = mean_squared_error(y2, y2_model, squared=False)
print("RMSE = " + str(rmse))

## 3.6 Model validation via cross-validation

One disadvantage of using a holdout set for model validation is that we have lost a portion of our data to the model training.
In the above case, half the dataset does not contribute to the training of the model!
This is not optimal, and can cause problems – especially if the initial set of training data is small.

One way to address this is to use *cross-validation*; that is, to do a sequence of fits where each subset of the data is used both as a training set and as a validation set.
Visually, it might look something like this:

In [ ]:
def draw_rects(N, ax, textprop={}):
    for i in range(N):
        ax.add_patch(plt.Rectangle((0, i), 5, 0.7, fc='black'))
        ax.add_patch(plt.Rectangle((5. * i / N, i), 5. / N, 0.7, fc='lightgray'))
        ax.text(5. * (i + 0.5) / N, i + 0.35,
                "validation\nset", ha='center', va='center', **textprop)
        ax.text(0, i + 0.35, "trial {0}".format(N - i),
                ha='right', va='center', rotation=90, **textprop)
    ax.set_xlim(-1, 6)
    ax.set_ylim(-0.2, N + 0.2)

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
draw_rects(2, ax, textprop=dict(size=14))

Here we do two validation trials, alternately using each half of the data as a holdout set.
Using the split data from before, we could implement it like this:

In [ ]:
# Let's go back to the 3-degree model
model = PolynomialRegression(3)

In [ ]:
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
print(mean_squared_error(y2, y2_model, squared=False), ',', 
      mean_squared_error(y1, y1_model, squared=False))

What comes out are two accuracy scores, which we could combine (by, say, taking the mean) to get a better measure of the global model performance.
This particular form of cross-validation is a *two-fold cross-validation*—that is, one in which we have split the data into two sets and used each in turn as a validation set.

We could expand on this idea to use even more trials, and more folds in the data—for example, here is a visual depiction of five-fold cross-validation:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
draw_rects(5, ax, textprop=dict(size=10))

Here we split the data into five groups, and use each of them in turn to evaluate the model fit on the other 4/5 of the data.
This would be rather tedious to do by hand, and so we can use Scikit-Learn's ``cross_val_score`` convenience routine to do it succinctly:

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

Repeating the validation across different subsets of the data gives us an even better idea of the performance of the algorithm.

Scikit-Learn implements a number of useful cross-validation schemes that are useful in particular situations; these are implemented via iterators in the ``cross_validation`` module.
For example, we might wish to go to the extreme case in which our number of folds is equal to the number of data points: that is, we train on all points but one in each trial.
This type of cross-validation is known as *leave-one-out* cross validation, and can be used as follows:

In [ ]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv = LeaveOneOut(), 
                         scoring='neg_root_mean_squared_error')
scores

Now let's go to a 4-degree polynomial and see how it performs

In [ ]:
model = PolynomialRegression(4)
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv = LeaveOneOut(), 
                         scoring='neg_root_mean_squared_error')
scores

Because we have 30 samples, the leave one out cross-validation yields scores for 30 trials, and the score indicates either small error or larger error.
Taking the mean of these gives an estimate of the error rate:

In [ ]:
scores.mean()

To look at this in another light, consider what happens if we use these two models to predict the y-value for some new data.
In the following diagrams, the red/lighter points indicate data that is omitted from the training set:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

X, y = make_data()
X2, y2 = make_data(10, rseed=42)

ax[0].scatter(X.ravel(), y, s=40, c='blue')
ax[0].plot(xfit.ravel(), model1.predict(xfit), color='gray')
ax[0].axis([-0.1, 1.0, -2, 14])
ax[0].set_title('High-bias model: Underfits the data', size=14)
ax[0].scatter(X2.ravel(), y2, s=40, c='red')
ax[0].text(0.02, 0.98, 
           "training score: $RMSE$ = {0:.2f}".format(mean_squared_error(model1.predict(X), y, squared=False)),
           ha='left', va='top', transform=ax[0].transAxes, size=14, color='blue')
ax[0].text(0.02, 0.91, 
           "training score: $RMSE$ = {0:.2f}".format(mean_squared_error(model1.predict(X2), y2, squared=False)),
           ha='left', va='top', transform=ax[0].transAxes, size=14, color='red')

ax[1].scatter(X.ravel(), y, s=40, c='blue')
ax[1].plot(xfit.ravel(), model20.predict(xfit), color='gray')
ax[1].axis([-0.1, 1.0, -2, 14])
ax[1].set_title('High-variance model: Overfits the data', size=14)
ax[1].scatter(X2.ravel(), y2, s=40, c='red')
ax[1].text(0.02, 0.98, 
           "training score: $RMSE$ = {0:.2g}".format(mean_squared_error(model20.predict(X), y, squared=False)),
           ha='left', va='top', transform=ax[1].transAxes, size=14, color='blue')
ax[1].text(0.02, 0.91, 
           "training score: $RMSE$ = {0:.2g}".format(mean_squared_error(model20.predict(X2), y2, squared=False)),
           ha='left', va='top', transform=ax[1].transAxes, size=14, color='red')

Let's also assess our models on the measurement of $R^2$, or [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination). This statistic measures how well a model performs relative to a simple mean of the target values. $R^2=1$ indicates a perfect match, $R^2=0$ indicates the model does no better than simply taking the mean of the data, and negative values mean even worse models.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

X, y = make_data()
X2, y2 = make_data(10, rseed=42)

ax[0].scatter(X.ravel(), y, s=40, c='blue')
ax[0].plot(xfit.ravel(), model1.predict(xfit), color='gray')
ax[0].axis([-0.1, 1.0, -2, 14])
ax[0].set_title('High-bias model: Underfits the data', size=14)
ax[0].scatter(X2.ravel(), y2, s=40, c='red')
ax[0].text(0.02, 0.98, "training score: $R^2$ = {0:.2f}".format(model1.score(X, y)),
           ha='left', va='top', transform=ax[0].transAxes, size=14, color='blue')
ax[0].text(0.02, 0.91, "validation score: $R^2$ = {0:.2f}".format(model1.score(X2, y2)),
           ha='left', va='top', transform=ax[0].transAxes, size=14, color='red')

ax[1].scatter(X.ravel(), y, s=40, c='blue')
ax[1].plot(xfit.ravel(), model20.predict(xfit), color='gray')
ax[1].axis([-0.1, 1.0, -2, 14])
ax[1].set_title('High-variance model: Overfits the data', size=14)
ax[1].scatter(X2.ravel(), y2, s=40, c='red')
ax[1].text(0.02, 0.98, "training score: $R^2$ = {0:.2g}".format(model20.score(X, y)),
           ha='left', va='top', transform=ax[1].transAxes, size=14, color='blue')
ax[1].text(0.02, 0.91, "validation score: $R^2$ = {0:.2g}".format(model20.score(X2, y2)),
           ha='left', va='top', transform=ax[1].transAxes, size=14, color='red')

From the scores associated with these two models, we can make an observation that holds more generally:

- For high-bias models, the performance of the model on the validation set is similar to the performance on the training set.
- For high-variance models, the performance of the model on the validation set is far worse than the performance on the training set.


Other cross-validation schemes can be used similarly.
For a description of what is available in Scikit-Learn, use IPython to explore the ``sklearn.cross_validation`` submodule, or take a look at Scikit-Learn's online [cross-validation documentation](http://scikit-learn.org/stable/modules/cross_validation.html).

If we imagine that we have some ability to tune the model complexity, we would expect the training score and validation score to behave as illustrated in the following figure:

In [ ]:
x = np.linspace(0, 1, 1000)
y1 = -(x - 0.5) ** 2
y2 = y1 - 0.33 + np.exp(x - 1)

fig, ax = plt.subplots()
ax.plot(x, y2, lw=10, alpha=0.5, color='blue')
ax.plot(x, y1, lw=10, alpha=0.5, color='red')

ax.text(0.15, 0.2, "training score", rotation=45, size=16, color='blue')
ax.text(0.2, -0.05, "validation score", rotation=20, size=16, color='red')

ax.text(0.02, 0.1, r'$\longleftarrow$ High Bias', size=18, rotation=90, va='center')
ax.text(0.98, 0.1, r'$\longleftarrow$ High Variance $\longrightarrow$', size=18, rotation=90, ha='right', va='center')
ax.text(0.48, -0.12, 'Best$\\longrightarrow$\nModel', size=18, rotation=90, va='center')

ax.set_xlim(0, 1)
ax.set_ylim(-0.3, 0.5)

ax.set_xlabel(r'model complexity $\longrightarrow$', size=14)
ax.set_ylabel(r'model score $\longrightarrow$', size=14)

ax.xaxis.set_major_formatter(plt.NullFormatter())
ax.yaxis.set_major_formatter(plt.NullFormatter())

ax.set_title("Validation Curve Schematic", size=16)

The diagram shown here is often called a *validation curve*, and we see the following essential features:

- The training score is everywhere higher than the validation score; the model will generally be a better fit to data it has seen than to data it has not seen.
- For very low model complexity (a high-bias model), the training data is under-fit, which means that the model is a poor predictor both for the training data and for any previously unseen data.
- For very high model complexity (a high-variance model), the training data is over-fit, which means that the model predicts the training data very well, but fails for any previously unseen data.
- For some intermediate value, the validation curve has a maximum. This level of complexity indicates a suitable trade-off between bias and variance.

The means of tuning the model complexity varies from model to model; when we discuss individual models in depth in later sections, we will see how each model allows for such tuning.

## 3.7 Validation curves in Scikit-Learn

Let's look at an example of using cross-validation to compute the validation curve for a class of models.
Here we will use a *polynomial regression* model: a generalized linear model in which the degree of the polynomial is a tunable parameter.
For example, a degree-1 polynomial fits a straight line to the data; for model parameters $a$ and $b$:

$$
y = ax + b
$$

A degree-3 polynomial fits a cubic curve to the data; for model parameters $a, b, c, d$:

$$
y = ax^3 + bx^2 + cx + d
$$

We can generalize this to any number of polynomial features.
In Scikit-Learn, we can implement this with a simple linear regression combined with the polynomial preprocessor.
We will use a *pipeline* to string these operations together .

The knob controlling model complexity in this case is the degree of the polynomial, which can be any non-negative integer.
A useful question to answer is this: what degree of polynomial provides a suitable trade-off between bias (under-fitting) and variance (over-fitting)?

We can make progress in this by visualizing the validation curve for this particular data and model; this can be done straightforwardly using the ``validation_curve`` convenience routine provided by Scikit-Learn.
Given a model, data, parameter name, and a range to explore, this function will automatically compute both the training score and validation score across the range:

In [ ]:
from sklearn.model_selection import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          'polynomialfeatures__degree', degree, cv=5)

plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

This shows precisely the qualitative behavior we expect: the training score is everywhere higher than the validation score; the training score is monotonically improving with increased model complexity; and the validation score reaches a maximum before dropping off as the model becomes over-fit.

From the validation curve, we can read-off that the optimal trade-off between bias and variance is found for a third-order polynomial; we can compute and display this fit over the original data as follows:

In [ ]:
plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = PolynomialRegression(5).fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim);

Notice that finding this optimal model did not actually require us to compute the training score, but examining the relationship between the training score and validation score can give us useful insight into the performance of the model.

## 3.8 EXTRA - The learning curve


We will duplicate the preceding code to plot the validation curve for this larger dataset; for reference let's over-plot the previous results as well:

In [ ]:
X2, y2 = make_data(200)
plt.scatter(X2.ravel(), y2);

In [ ]:
degree = np.arange(21)
train_score2, val_score2 = validation_curve(PolynomialRegression(), X2, y2,
                                            'polynomialfeatures__degree', degree, cv=5)

plt.plot(degree, np.median(train_score2, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score2, 1), color='red', label='validation score')
plt.plot(degree, np.median(train_score, 1), color='blue', alpha=0.3, linestyle='dashed')
plt.plot(degree, np.median(val_score, 1), color='red', alpha=0.3, linestyle='dashed')
plt.legend(loc='lower center')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

The solid lines show the new results, while the fainter dashed lines show the results of the previous smaller dataset.
It is clear from the validation curve that the larger dataset can support a much more complicated model: the peak here is probably around a degree of 6, but even a degree-20 model is not seriously over-fitting the data—the validation and training scores remain very close.

Thus we see that the behavior of the validation curve has not one but two important inputs: the model complexity and the number of training points.
It is often useful to to explore the behavior of the model as a function of the number of training points, which we can do by using increasingly larger subsets of the data to fit our model.
A plot of the training/validation score with respect to the size of the training set is known as a *learning curve.*

The general behavior we would expect from a learning curve is this:

- A model of a given complexity will *overfit* a small dataset: this means the training score will be relatively high, while the validation score will be relatively low.
- A model of a given complexity will *underfit* a large dataset: this means that the training score will decrease, but the validation score will increase.
- A model will never, except by chance, give a better score to the validation set than the training set: this means the curves should keep getting closer together but never cross.

With these features in mind, we would expect a learning curve to look qualitatively like that shown in the following figure:

In [ ]:
N = np.linspace(0, 1, 1000)
y1 = 0.75 + 0.2 * np.exp(-4 * N)
y2 = 0.7 - 0.6 * np.exp(-4 * N)

fig, ax = plt.subplots()
ax.plot(x, y1, lw=10, alpha=0.5, color='blue')
ax.plot(x, y2, lw=10, alpha=0.5, color='red')

ax.text(0.2, 0.88, "training score", rotation=-10, size=16, color='blue')
ax.text(0.2, 0.5, "validation score", rotation=30, size=16, color='red')

ax.text(0.98, 0.45, r'Good Fit $\longrightarrow$', size=18, rotation=90, ha='right', va='center')
ax.text(0.02, 0.57, r'$\longleftarrow$ High Variance $\longrightarrow$', size=18, rotation=90, va='center')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel(r'training set size $\longrightarrow$', size=14)
ax.set_ylabel(r'model score $\longrightarrow$', size=14)

ax.xaxis.set_major_formatter(plt.NullFormatter())
ax.yaxis.set_major_formatter(plt.NullFormatter())

ax.set_title("Learning Curve Schematic", size=16)

The notable feature of the learning curve is the convergence to a particular score as the number of training samples grows.
In particular, once you have enough points that a particular model has converged, *adding more training data will not help you!*
The only way to increase model performance in this case is to use another (often more complex) model.

### 3.8.1 Learning curves in Scikit-Learn

Scikit-Learn offers a convenient utility for computing such learning curves from your models; here we will compute a learning curve for our original dataset with a second-order polynomial model and a ninth-order polynomial:

In [ ]:
from sklearn.model_selection import learning_curve

fig, ax = plt.subplots(1, 2, figsize=(16, 6))

X,y = make_data(N=40)

for i, degree in enumerate([2, 5]):
    N, train_lc, val_lc = learning_curve(PolynomialRegression(degree),
                                         X, y, cv=7,
                                         train_sizes=np.linspace(0.3, 1, 25))

    ax[i].plot(N, np.mean(train_lc, 1), color='blue', label='training score')
    ax[i].plot(N, np.mean(val_lc, 1), color='red', label='validation score')
    ax[i].hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0], N[-1],
                 color='gray', linestyle='dashed')

    ax[i].set_ylim(0, 1.1)
    ax[i].set_xlim(N[0], N[-1])
    ax[i].set_xlabel('training size')
    ax[i].set_ylabel('score')
    ax[i].set_title('degree = {0}'.format(degree), size=14)
    ax[i].legend(loc='best')

This is a valuable diagnostic, because it gives us a visual depiction of how our model responds to increasing training data.
In particular, when your learning curve has already converged (i.e., when the training and validation curves are already close to each other) *adding more training data will not significantly improve the fit!*
This situation is seen in the left panel, with the learning curve for the degree-2 model.

The only way to increase the converged score is to use a different (usually more complicated) model.
We see this in the right panel: by moving to a much more complicated model, we increase the score of convergence (indicated by the dashed line), but at the expense of higher model variance (indicated by the difference between the training and validation scores).
If we were to add even more data points, the learning curve for the more complicated model would eventually converge.

Plotting a learning curve for your particular choice of model and dataset can help you to make this type of decision about how to move forward in improving your analysis.

## 3.9 EXTRA - Validation in Practice: Grid Search

The preceding discussion is meant to give you some intuition into the trade-off between bias and variance, and its dependence on model complexity and training set size.
In practice, models generally have more than one knob to turn, and thus plots of validation and learning curves change from lines to multi-dimensional surfaces.
In these cases, such visualizations are difficult and we would rather simply find the particular model that maximizes the validation score.

Scikit-Learn provides automated tools to do this in the grid search module.
Here is an example of using grid search to find the optimal polynomial model.
We will explore a three-dimensional grid of model features; namely the polynomial degree, the flag telling us whether to fit the intercept, and the flag telling us whether to normalize the problem.
This can be set up using Scikit-Learn's ``GridSearchCV`` meta-estimator:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False],
              'linearregression__normalize': [True, False]}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)

Notice that like a normal estimator, this has not yet been applied to any data.
Calling the ``fit()`` method will fit the model at each grid point, keeping track of the scores along the way:

In [ ]:
grid.fit(X, y);

Now that this is fit, we can ask for the best parameters as follows:

In [ ]:
grid.best_params_

Finally, if we wish, we can use the best model and show the fit to our data using code from before:

In [ ]:
model = grid.best_estimator_

plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = model.fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test)
plt.axis(lim);

The grid search provides many more options, including the ability to specify a custom scoring function, to parallelize the computations, to do randomized searches, and more.
For information, see the examples in Scikit-Learn's [grid search documentation](http://Scikit-Learn.org/stable/modules/grid_search.html).

## 3.10 Summary

In this section, we have begun to explore the concept of **model validation** and **hyperparameter optimization**, focusing on intuitive aspects of the bias–variance trade-off and how it comes into play when fitting models to data.

In particular, we found that the use of a **validation set** or **cross-validation** approach is *vital* when tuning parameters in order to avoid over-fitting.

Next, we want to show you how you can incorporate even more sophisticated statistical software into your research methods, straight from `R`! If you're not familiar with `R`, there are some slight syntax differences, but hopefully the code doesn't look too strange. 

# Section 4: Mixing methods in Python and R

In this section we'll learn how to use **both** R and Python in your data analysis through the tool of the `rpy2` package.

*rpy2* is running an embedded R, providing access to it from Python using R’s own C-API through either:

* a high-level interface making R functions an objects just like Python functions and providing a seamless conversion to numpy and pandas data structures
* a low-level interface closer to the C-API

It allows us to intertwine `R` and `Python` together in the same notebook. It is also providing features for when working with jupyter notebooks or ipython.

In [ ]:
%%capture

%load_ext rpy2.ipython
from rpy2.robjects.packages import importr
utils = importr('utils')
### this install takes a second. it looks like it is hanging up, but then it works ###
utils.install_packages('ggplot2',  repos = 'https://cloud.r-project.org')
utils.install_packages('tidyverse',  repos = 'https://cloud.r-project.org')

## 4.1 Turning a cell into R

We can turn a line of code into `R` using the `%R` cell magic command. We can turn an *entire code cell* into `R` by putting the cell magic command `%%R` at the top of the cell (notice the double `%`s).

In [ ]:
v_ex_py = np.array([1, 2, 3, 4, 5])
%R v_ex_r <- c(1, 2, 3, 4, 5) # Set a line to R

In [ ]:
# For referecne, uncomment the line below to try R syntax without specifying %R

v_ex_r <- c(1, 2, 3, 4, 5) 

In [ ]:
%%R # Set the whole cell to R

x_vals <- c(1, 2, 3, 4, 5)
y_vals <- c(1, 2, 4, 8, 16)

plot(x_vals, y_vals, col='purple', pch=12, main='R plot in Python')

## 4.2 Passing data back and forth

We can pass variables between the `R` and `Python` cells by using the `%%R -i` and `%%R -o` commands.

In [ ]:
df_AB = pd.DataFrame({'A':(np.random.randint(-2, 2, 10)+np.arange(10)).tolist(),
                      'B':(np.random.randint(-2, 2, 10)+np.arange(10)).tolist()})
df_AB.head()

Let's pass this `DataFrame` into an `R` cell and plot it. 

In [ ]:
%%R -i df_AB

plot(df_AB, main='Plotting a Python DF in R', col='purple', pch=9)

We can also output objects from `R` into the `Python` environment. 

In [ ]:
%%R -o model,coef 
          # ^ Do not leave a space between your commas
model <- lm(B ~ A, data = df_AB)
coef <- model$coefficients

In [ ]:
print(model)
print(type(model))

print(coef)
print(type(coef))

print(list(coef))

With the use of both of these environments, we can take advantage of what both langauges have to offer. For instance, we can take a dataset from `R` and throw it into our `Python` environment to plot it up with a familiar `seaborn` command.

In [ ]:
%%R -o cars_df

library(datasets)
cars_df <- cars

In [ ]:
import seaborn as sns
print(cars_df.head())

sns.pairplot(x_vars=['speed'], y_vars=['dist'], data=cars_df, height=5)
plt.show()

In [ ]:
%%R 
r_var <- c(1,2,3,4,5)

We can also pass variables back and forth with the single-line `%Rget` and `%Rpush` commands. These commands are one line only (notice the single % sign), so the rest of the cell stays in the `Python` environment. 

In [ ]:
python_var = %Rget r_var

print(python_var)
print(type(python_var))
print(list(python_var))

In [ ]:
my_python_var1 = np.array([1, 2, 3, 4, 5])
my_python_var2 = np.array([1, 2, 4, 8, 16])

%Rpush my_python_var1 my_python_var2

In [ ]:
%%R

plot(my_python_var1, my_python_var2, col='purple', 
     pch=16, main='Python objects pushed to R')

## 4.3 `R` libraries

With this package, we can also take advantage of the landscape of libraries that `R` offers.

In [ ]:
## Import Librarys

%%capture
%%R
library(tidyverse)
library(broom)

In [ ]:
df = pd.DataFrame({'Letter':['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'],
                      'X':(np.random.randint(1, 10, 9)).tolist(),
                      'Y':(np.random.randint(1, 10, 9)).tolist(),
                      'Z':(np.random.randint(1, 10, 9)).tolist()})

Let's use the famous `ggplot`.

In [ ]:
%%R -i df
ggplot(data = df) + geom_point(aes(x=X, y=Y, color=Letter, size=Z))

In [ ]:
x = np.array([1, 2, 4, 6, 5, 8])
y = np.array([0, 1, 3, 2, 5, 7])

We can also use the regression libraries that make `R` such a popular language in data science.

In [ ]:
%%R -i x,y -o my_coef

xylm = lm(y~x)
my_coef = coef(xylm)
par(mfrow = c(2, 2))
plot(xylm)

In [ ]:
print(my_coef)

Now let's try a more advanced example using the Gapminder data we visualized yesterday.

## EXTRA - 4.4 Advanced example: Gapminder data

First, we'll load the data again.

In [ ]:
gapminder = px.data.gapminder()
px.scatter(gapminder, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="country", hover_name="country", 
           log_x = True, 
           size_max=45, range_x=[100,100000], range_y=[25,90])

### 4.4.1 Stardard Ordinary Least Squares (OLS) model from `R`

Let's use the popular OLS library from `R` to create a linear model, like we did earlier with Sci-KitLearn.

In [ ]:
%%R -i gapminder
m1.ols <- lm(lifeExp ~ country + gdpPercap + pop, data = gapminder)
summary(m1.ols)$coefficients[c('gdpPercap', 'pop'),]

Notice that with this `R` package we are able to get statistics on our coefficients, something that we couldn't do in SciKitLearn. 



### 4.4.3 Now let's use a different library, and include the `lfe` module 

**Note** (standard errors are different)

In [ ]:
%%capture
utils.install_packages('lfe',  repos = 'https://cloud.r-project.org')
%R library(lfe)

In [ ]:
%%R
m1.lfe <- felm(lifeExp ~ gdpPercap + pop | country, data = gapminder)

std.errs <- cbind(
  summary(m1.ols)$coefficients[c('gdpPercap', 'pop'),2],
  summary(m1.lfe)$coefficients[,2]
)
colnames(std.errs) <- c('OLS w/ Intercepts', 'felm Model')
std.errs

### 4.4.4 And now with the `plm` module 

In [ ]:
%%capture
utils.install_packages('plm',  repos = 'https://cloud.r-project.org')
%R library(plm)

In [ ]:
%%R
m1.plm <- plm(lifeExp ~ gdpPercap + pop, data = gapminder, model = 'within', index = c('country'))

summary(m1.plm)

## 4.5 Summary

We made it through another day!

Let's summarize. Today we've learned:

* How to use SciKitLearn to create regression models in Python
* The choise of model **and** hyperparameters is crucial to determining the "best" model for your data
* Validation set and cross-validation are an invaluable test for machine learning.
* There are advantages to using `Python` and `R`, and we can get the best of both worlds by combining them in out data analysis practices.

Obviously we couldn't teach it all in this one class, so here are some more links to help learn about the capabilities of the SciKitLearn and `rpy2` packages.

**Links used**
* https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
* https://blog.revolutionanalytics.com/2016/01/pipelining-r-python.html

**`rpy2` documentation**: 
* https://rpy2.github.io/doc/latest/html/index.html
* https://rpy2.github.io/doc/v3.0.x/html/interactive.html#module-rpy2.ipython.rmagic

**R-magic documentation** 
* https://ipython.org/ipython-doc/2/config/extensions/rmagic.html

--------------------
--------------------
--------------------

# Section 5: EXTRAS



**Warning**: The following sections are in no particular order, but are some extra sections if you're interested in learning more about SciKitLearn and linear models in Python.

## 5.1 Example: Predicting Bicycle Traffic

As an example, let's take a look at whether we can predict the number of bicycle trips across Seattle's Fremont Bridge based on weather, season, and other factors.

In this section, we will join the bike data with another dataset, and try to determine the extent to which weather and seasonal factors—temperature, precipitation, and daylight hours—affect the volume of bicycle traffic through this corridor.
We will perform a simple linear regression to relate weather and other information to bicycle counts, in order to estimate how a change in any one of these parameters affects the number of riders on a given day.

In particular, this is an example of how the tools of Scikit-Learn can be used in a statistical modeling framework, in which the parameters of the model are assumed to have interpretable meaning.
As discussed previously, this is not a standard approach within machine learning, but such interpretation is possible for some models.

Let's start by loading the two datasets, indexing by date:

In [ ]:
counts = pd.read_csv(loaddir+'/Fremont_Bridge_Hourly_Bicycle_Counts_by_Month_October_2012_to_present.csv', index_col='Date', parse_dates=True)
weather = pd.read_csv(loaddir+'/bike_weather.csv', index_col = 'DATE', parse_dates=True)

In [ ]:
daily = counts.resample('d').sum()
daily.head()

Next we will compute the total daily bicycle traffic, and put this in its own dataframe:

In [ ]:
daily = counts.resample('d').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # remove other columns

We saw previously that the patterns of use generally vary from day to day; let's account for this in our data by adding binary columns that indicate the day of the week:

In [ ]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)

Similarly, we might expect riders to behave differently on holidays; let's add an indicator of this as well:

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
daily['holiday'].fillna(0, inplace=True)

We also might suspect that the hours of daylight would affect how many people ride; let's use the standard astronomical calculation to add this information:

In [ ]:
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - pd.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))
daily[['daylight_hrs']].plot()
plt.ylim(8, 17)

We can also add the average temperature and total precipitation to the data.
In addition to the inches of precipitation, let's add a flag that indicates whether a day is dry (has zero precipitation):

In [ ]:
# temperatures are in 1/10 deg C; convert to C
weather['TMIN'] /= 10
weather['TMAX'] /= 10
weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])

# precip is in 1/10 mm; convert to inches
weather['PRCP'] /= 254
weather['dry day'] = (weather['PRCP'] == 0).astype(int)

daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']])
daily.tail()

Finally, let's add a counter that increases from day 1, and measures how many years have passed.
This will let us measure any observed annual increase or decrease in daily crossings:

In [ ]:
daily['annual'] = (daily.index - daily.index[0]).days / 365.

Now our data is in order, and we can take a look at it:

In [ ]:
daily.head()

With this in place, we can choose the columns to use, and fit a linear regression model to our data.
We will set ``fit_intercept = False``, because the daily flags essentially operate as their own day-specific intercepts:

In [ ]:
# Drop any rows with null values
daily.dropna(axis=0, how='any', inplace=True)

column_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'holiday',
                'daylight_hrs', 'PRCP', 'dry day', 'Temp (C)', 'annual']
X = daily[column_names]
y = daily['Total']

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
daily['predicted'] = model.predict(X)

Finally, we can compare the total and predicted bicycle traffic visually:

In [ ]:
daily[['Total', 'predicted']].plot(alpha=0.5);

It is evident that we have missed some key features, especially during the summer time.
Either our features are not complete (i.e., people decide whether to ride to work based on more than just these) or there are some nonlinear relationships that we have failed to take into account (e.g., perhaps people ride less at both high and low temperatures).
Nevertheless, our rough approximation is enough to give us some insights, and we can take a look at the coefficients of the linear model to estimate how much each feature contributes to the daily bicycle count:

In [ ]:
params = pd.Series(model.coef_, index=X.columns)
params

These numbers are difficult to interpret without some measure of their uncertainty.
We can compute these uncertainties quickly using bootstrap resamplings of the data:

In [ ]:
from sklearn.utils import resample
np.random.seed(1)
err = np.std([model.fit(*resample(X, y)).coef_
              for i in range(1000)], 0)

With these errors estimated, let's again look at the results:

In [ ]:
print(pd.DataFrame({'effect': params.round(0),
                    'error': err.round(0)}))

We first see that there is a relatively stable trend in the weekly baseline: there are many more riders on weekdays than on weekends and holidays.
We see that for each additional hour of daylight, 129 ± 9 more people choose to ride; a temperature increase of one degree Celsius encourages 65 ± 4 people to grab their bicycle; a dry day means an average of 548 ± 33 more riders, and each inch of precipitation means 665 ± 62 more people leave their bike at home.
Once all these effects are accounted for, we see a modest increase of 27 ± 18 new daily riders each year.

Our model is almost certainly missing some relevant information. For example, nonlinear effects (such as effects of precipitation *and* cold temperature) and nonlinear trends within each variable (such as disinclination to ride at very cold and very hot temperatures) cannot be accounted for in this model.
Additionally, we have thrown away some of the finer-grained information (such as the difference between a rainy morning and a rainy afternoon), and we have ignored correlations between days (such as the possible effect of a rainy Tuesday on Wednesday's numbers, or the effect of an unexpected sunny day after a streak of rainy days).
These are all potentially interesting effects, and you now have the tools to begin exploring them if you wish!

## 5.2 Preprocessing your data
* https://scikit-learn.org/stable/modules/preprocessing.html
* https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
* https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html


## 5.3 Basis functions and Gaussian models

Of course, other basis functions are possible.
For example, one useful pattern is to fit a model that is not a sum of polynomial bases, but a sum of Gaussian bases.
The result might look something like the following figure:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for one-dimensional input"""
    
    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor
    
    @staticmethod
    def _gauss_basis(x, y, width, axis=None):
        arg = (x - y) / width
        return np.exp(-0.5 * np.sum(arg ** 2, axis))
        
    def fit(self, X, y=None):
        # create N centers spread along the data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self
        
    def transform(self, X):
        return self._gauss_basis(X[:, :, np.newaxis], self.centers_,
                                 self.width_, axis=1)

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)
xfit = np.linspace(0, 10, 1000)

gauss_model = make_pipeline(GaussianFeatures(10, 1.0),LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

gf = gauss_model.named_steps['gaussianfeatures']
lm = gauss_model.named_steps['linearregression']

fig, ax = plt.subplots()

for i in range(10):
    selector = np.zeros(10)
    selector[i] = 1
    Xfit = gf.transform(xfit[:, None]) * selector
    yfit = lm.predict(Xfit)
    ax.fill_between(xfit, yfit.min(), yfit, color='gray', alpha=0.2)

ax.scatter(x, y)
ax.plot(xfit, gauss_model.predict(xfit[:, np.newaxis]))
ax.set_xlim(0, 10)
ax.set_ylim(yfit.min(), 1.5)

In [ ]:
x.shape

The shaded regions in the plot are the scaled basis functions, and when added together they reproduce the smooth curve through the data.
These Gaussian basis functions are not built into Scikit-Learn, but we can write a custom transformer that will create them, as shown here and illustrated in the following figure (Scikit-Learn transformers are implemented as Python classes; reading Scikit-Learn's source is a good way to see how they can be created):

In [ ]:
gauss_model = make_pipeline(GaussianFeatures(20),LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit)
plt.xlim(0, 10);

We put this example here just to make clear that there is nothing magic about polynomial basis functions: if you have some sort of intuition into the generating process of your data that makes you think one basis or another might be appropriate, you can use them as well.

## 5.4 Regularization

The introduction of basis functions into our linear regression makes the model much more flexible, but it also can very quickly lead to over-fitting.
For example, if we choose too many Gaussian basis functions, we end up with results that don't look so good:

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)
xfit = np.linspace(0, 10, 1000)

In [ ]:
model = make_pipeline(GaussianFeatures(30), LinearRegression())
model.fit(x[:, np.newaxis], y)

plt.scatter(x, y)
plt.plot(xfit, model.predict(xfit[:, np.newaxis]))

plt.xlim(0, 10)
plt.ylim(-1.5, 1.5);

With the data projected to the 30-dimensional basis, the model has far too much flexibility and goes to extreme values between locations where it is constrained by data.
We can see the reason for this if we plot the coefficients of the Gaussian bases with respect to their locations:

In [ ]:
def basis_plot(model, title=None):
    fig, ax = plt.subplots(2, sharex=True)
    model.fit(x[:, np.newaxis], y)
    ax[0].scatter(x, y)
    ax[0].plot(xfit, model.predict(xfit[:, np.newaxis]))
    ax[0].set(xlabel='x', ylabel='y', ylim=(-1.5, 1.5))
    
    if title:
        ax[0].set_title(title)

    ax[1].scatter(model.steps[0][1].centers_,
               model.steps[1][1].coef_)
    ax[1].set(xlabel='basis location',
              ylabel='coefficient',
              xlim=(0, 10))
    
model = make_pipeline(GaussianFeatures(30), LinearRegression())
basis_plot(model)

The lower panel of this figure shows the amplitude of the basis function at each location.
This is typical over-fitting behavior when basis functions overlap: the coefficients of adjacent basis functions blow up and cancel each other out.
We know that such behavior is problematic, and it would be nice if we could limit such spikes expliticly in the model by penalizing large values of the model parameters.
Such a penalty is known as *regularization*, and comes in several forms.

### 5.4.1 Ridge regression ($L_2$ Regularization)

Perhaps the most common form of regularization is known as *ridge regression* or $L_2$ *regularization*, sometimes also called *Tikhonov regularization*.
This proceeds by penalizing the sum of squares (2-norms) of the model coefficients; in this case, the penalty on the model fit would be 
$$
P = \alpha\sum_{n=1}^N \theta_n^2
$$
where $\alpha$ is a free parameter that controls the strength of the penalty.
This type of penalized model is built into Scikit-Learn with the ``Ridge`` estimator:

In [ ]:
from sklearn.linear_model import Ridge
model = make_pipeline(GaussianFeatures(30), Ridge(alpha=100))
basis_plot(model, title='Ridge Regression')

The $\alpha$ parameter is essentially a knob controlling the complexity of the resulting model.
In the limit $\alpha \to 0$, we recover the standard linear regression result; in the limit $\alpha \to \infty$, all model responses will be suppressed.
One advantage of ridge regression in particular is that it can be computed very efficiently—at hardly more computational cost than the original linear regression model.

### 5.4.2 Lasso regression ($L_1$ regularization)

Another very common type of regularization is known as lasso, and involves penalizing the sum of absolute values (1-norms) of regression coefficients:
$$
P = \alpha\sum_{n=1}^N |\theta_n|
$$
Though this is conceptually very similar to ridge regression, the results can differ surprisingly: for example, due to geometric reasons lasso regression tends to favor *sparse models* where possible: that is, it preferentially sets model coefficients to exactly zero.

We can see this behavior in duplicating the ridge regression figure, but using L1-normalized coefficients:

In [ ]:
from sklearn.linear_model import Lasso
model = make_pipeline(GaussianFeatures(30), Lasso(alpha=0.1))
basis_plot(model, title='Lasso Regression')

With the lasso regression penalty, the majority of the coefficients are exactly zero, with the functional behavior being modeled by a small subset of the available basis functions.
As with ridge regularization, the $\alpha$ parameter tunes the strength of the penalty, and should be determined via, for example, cross-validation.

## 5.5 Supervised learning example: Iris classification

Let's take a look at another example of this process, using the Iris dataset we discussed earlier.
Our question will be this: given a model trained on a portion of the Iris data, how well can we predict the remaining labels?

For this task, we will use an extremely simple generative model known as Gaussian naive Bayes, which proceeds by assuming each class is drawn from an axis-aligned Gaussian distribution.
Because it is so fast and has no hyperparameters to choose, Gaussian naive Bayes is often a good model to use as a baseline classification, before exploring whether improvements can be found through more sophisticated models.

We would like to evaluate the model on data it has not seen before, and so we will split the data into a *training set* and a *testing set*.
This could be done by hand, but it is more convenient to use the ``train_test_split`` utility function:

In [ ]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

In [ ]:
X_iris = iris.drop('species', axis=1)
X_iris.shape

In [ ]:
y_iris = iris['species']
y_iris.shape

In [ ]:
from sklearn.model_selection import train_test_split
# set the random state so you ge reproducible splits
Xtrain, Xtest, ytrain, ytest = train_test_split(X_iris, y_iris, random_state=1)

With the data arranged, we can follow our recipe to predict the labels:

In [ ]:
# 1. choose model class
from sklearn.naive_bayes import GaussianNB
# 3. instantiate model
model = GaussianNB()      
# 4. fit model to data                 
model.fit(Xtrain, ytrain)       
# 5. predict on new data           
y_model = model.predict(Xtest) 

In [ ]:
y_model

We can also use the ``partial_fit`` method multiple times if our data is too big and needs to be passed in chunks. 

Finally, we can use the ``accuracy_score`` utility to see the fraction of predicted labels that match their true value:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, y_model)

With an accuracy topping 97%, we see that even this very naive classification algorithm is effective for this particular dataset!

## 5.6 Unsupervised learning example: Iris dimensionality

As an example of an unsupervised learning problem, let's take a look at reducing the dimensionality of the Iris data so as to more easily visualize it.
Recall that the Iris data is four dimensional: there are four features recorded for each sample.

The task of dimensionality reduction is to ask whether there is a suitable lower-dimensional representation that retains the essential features of the data.
Often dimensionality reduction is used as an aid to visualizing data: after all, it is much easier to plot data in two dimensions than in four dimensions or higher!

Here we will use principal component analysis (PCA), which is a fast linear dimensionality reduction technique.
We will ask the model to return two components—that is, a two-dimensional representation of the data.

Following the sequence of steps outlined earlier, we have:

In [ ]:
from sklearn.decomposition import PCA  # 1. Choose the model class
model = PCA(n_components=2)            # 2. Instantiate the model with hyperparameters
model.fit(X_iris)                      # 3. Fit to data. Notice y is not specified!
X_2D = model.transform(X_iris)         # 4. Transform the data to two dimensions

iris['PCA1'] = X_2D[:, 0]
iris['PCA2'] = X_2D[:, 1]

px.scatter(
    iris,
    x = "PCA1", 
    y = "PCA2",
    color = 'species',
    width = 600,
    height = 400,
    template = 'plotly_dark',
    )


We see that in the two-dimensional representation, the species are fairly well separated, even though the PCA algorithm had no knowledge of the species labels!
This indicates to us that a relatively straightforward classification will probably be effective on the dataset, as we saw before.